# Predict ligand profiling using `kissim`

## Aim of this notebook

In order to asses the predictive power of `kissim`, we here choose a ligand-centric evaluation. We will compare if `kissim` can predict on- and off-targets determined in ligand profiling studies.

__Data__

- __Ligand-kinase profiling__: Karaman dataset downloaded from KinMap
- __Kinase-kinase profiling__: `kissim` dataset (kinase similarities)
- __Kinase names__: KinMap kinase names (maps kinase synonyms to main kinase names used in KinMap)
- __Ligand names__: PKIDB ligand names (maps ligand synonyms to main ligand names defined in PKIDB)

__Method__

- Prepare datasets
  - PDKIDB dataset: Ligand names
  - KinMap dataset: Kinase names
  - Karaman dataset: Ligand-kinase profiling data downloaded from KinMap (keep only ligands whose name can be cast to PKIDB name)
  - `kissim` dataset: Kinase-kinase profiling data (keep only kinases whose name can be cast to KinMap name)
  - Define ligands' on-targets as listed in PKIDB (keep only on-targets whose name can be cast to KinMap name)
- Merge datasets into "ligand dataset" per ligand; keep "ligand dataset" if enough coverage
  - "Ligand dataset" consists of
    - Ligand profiling data (Karaman et al. activities for ligand)
    - Kinase similarity data (`kissim` distances for ligand's on-target)
  - Coverage of a "ligand dataset" is high enough if the number of targets that fullfil the following conditions is greater than `MIN_N_TARGETS`:
    - Target is active w.r.t. to the ligand (in Karaman dataset; `KD_CUTOFF` defines activity)
    - Target is structurally covered in `kissim`
  - Some ligands have multiple on-targets; set up "ligand dataset" per ligand/on-target pair each
- Calculate `kissim` performance; measured using an enrichment factor within the top x% of ranked `kissim` results ($EF{_x\%}$)
  - Kinase is defined as active if Karaman et. al profiling data reports $K_d$ <= `KD_CUTOFF`
  - $EF{_x\%}$ is the ratio between _observed_ active kinases and the _theoretical maximum_ of active kinases in the top x% of ranked `kissim` results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Markdown

from src.data import ligands, kinases, profiling, distances, targets
from src.evaluation import ranks

In [ ]:
plt.style.use("seaborn")

In [ ]:
HERE = Path(_dh[-1])  # noqa: F821
DATA = HERE / "../../data"
RESULTS = HERE / "../../results"

In [ ]:
KD_CUTOFF = 100
MIN_N_TARGETS = 3

## Prepare datasets

We want to compare on- and off-targets seen in (1) the Karaman kinase-ligand profiling dataset and (2) the `kissim` kinase-kinase profiling dataset. Kinase names and ligand names must be homogenized; we will use kinase names from KinMap and ligand names from the PKIDB.

### Karaman dataset: Kinase-ligand profiling data

__Note__: Filters out ligands whose names cannot be mapped onto PKIDB ligand names.

#### Load full dataset (not needed)

In [ ]:
karaman_df = profiling.karaman()
print(f"Number of kinases: {karaman_df.shape[0]}")
print(f"Number of ligands: {karaman_df.shape[1]}")
karaman_df.head()

#### Load dataset with FDA-approved PKIDB ligands only

In [ ]:
karaman_df = profiling.karaman(pkidb_ligands=True, fda_approved=True)
print(f"Number of kinases: {karaman_df.shape[0]}")
print(f"Number of ligands: {karaman_df.shape[1]}")
karaman_df.head()

In [ ]:
karaman_df.notna().sum()

### `kissim` dataset: Kinase-kinase profiling data

__Note__: Filters out kinases whose names cannot be mapped to KinMap kinase names.

In [ ]:
distances_path = RESULTS / "../results/20210324/fingerprint_distances_41-41-41-41-41-41-41-41-83-83-83-83-111-111-111.json"

#### Load full dataset (not needed)

In [ ]:
kissim_df = distances.kissim(distances_path, structure_kinase_mapping_by="minimum", kinmap_kinases=False)
print(f"Kinase distance matrix shape: {kissim_df.shape}")

#### Load dataset with KinMap kinases only

In [ ]:
kissim_df = distances.kissim(distances_path, structure_kinase_mapping_by="minimum", kinmap_kinases=True)
print(f"Kinase distance matrix shape: {kissim_df.shape}")
kissim_df.iloc[:5, :5]

### Ligands' on-targets (PKIDB dataset)

__Note__: Filters out ligands whose targets are not available in PKIDB.

In [ ]:
pd.set_option("max_colwidth", 100)
ligand_targets = targets._pkidb_ligands_to_targets(karaman_df.columns, fda_approved=False)
ligand_targets

Ligand targets listed in the DrugBank or PKIDB are including reported on- and off-targets. For the purpose of this notebook, we would like to identify the intended targets (on-targets) only. This seems to be a not-so-easy task and involves a manual assignment based on literature search.

We will assign main on-targets manually based on the following table including
- Main on-targets reported in literature
- Availability of profiling data (with <= 100nM activity) for a given ligand-kinase combo in the Karaman profiling dataset 
- Use KinMap kinase names (since we are using the Karaman dataset provided by KinMap)
- For multitargeted ligands define multiple targets

| Approved kinase inhibitor                            | Intended kinase target (\* in Karaman profiling dataset <= 100nM) |
|------------------------------------------------------|---------------------------------------------------------|
| [Erlotinib](https://go.drugbank.com/drugs/DB00530)   | EGFR\* |
| [Gefitinib](https://go.drugbank.com/drugs/DB00317)   | EGFR\* |
| [Lapatinib](https://go.drugbank.com/drugs/DB01259)   | EGFR\*, ERBB2\* |
| [Tofacitinib](https://go.drugbank.com/drugs/DB08895) | JAK1, JAK2, JAK3 |
| [Vandetanib](https://go.drugbank.com/drugs/DB05294)  | VEGFR2, EGFR, RET; [see paper](https://www.ema.europa.eu/en/documents/assessment-report/caprelsa-epar-public-assessment-report_en.pdf) |
| [Imatinib](https://go.drugbank.com/drugs/DB00619)    | multitargeted inhibitor - TK: ABL1\*, KIT\*, PDGFRa\*; [see paper](https://www.hindawi.com/journals/cherp/2014/357027/) |
| [Sunitinib](https://go.drugbank.com/drugs/DB01268)   | multitargeted inhibitor - TK: VEGFR1/2, PDFGRa/b, KIT, FLT2, RET, CSF1R; [see paper](https://link.springer.com/article/10.2165/11318860-000000000-00000) |
| [Dasatinib](https://go.drugbank.com/drugs/DB01254)   | multitargeted inhibitor - TK: ABL1 (BCRABL), SRC, Eph\_ (Ephrins), \_GFR (GFR); see DrugBank Identification > Description |
| [Sorafenib](https://go.drugbank.com/drugs/DB00398)   | multitargeted inhibitor - TK/TKL (Raf/Mek/Erk pathways): RAF\_ (Raf) \[TKL\], PDFGR\_ (PDFG), VEGF2/3, KIT; see DrugBank Identification > Description |

Full ligand-kinase list:

In [ ]:
ligand_targets_list = ligand_targets[["ligand.input", "targets.kinmap"]]
ligand_targets_list = ligand_targets_list.to_numpy().tolist()
ligand_targets_list

## Merge datasets into "ligand datasets"

We will merge now profiling (Karaman et al.) and kinase similarity (`kissim`) information into one dataset per ligand (ligand dataset). 

We will only keep ligand datasets if we like the coverage of active kinases covered by `kissim` (Karaman et al.)

In [ ]:
for ligand_name, target_list in ligand_targets_list:
    for kinase_name in target_list:
        ranks.profiling_vs_similarity_dataset(
            kinase_name,
            kissim_df,
            "kissim",
            ligand_name,
            karaman_df,
            "karaman",
        ).head(5)

In [ ]:
ranks.profiling_vs_similarity_dataset(
    "EGFR",
    kissim_df,
    "kissim",
    "Erlotinib",
    karaman_df,
    "karaman",
)

In [ ]:
def create_ligand_dataset(karaman_df, kissim_df, ligand_name, kinase_name, kd_cutoff):
    
    ligand_dataset = pd.DataFrame(
        {
            "kinase.kinmap_name": karaman_df[ligand_name].index.to_list(),
            "activity.kd_nm": karaman_df[ligand_name].to_list()
        }
    )
    # Sort by Kd and set activity rank accordingly
    #ligand_dataset = ligand_dataset.sort_values(by="activity.kd_nm").reset_index(drop=True)
    #ligand_dataset["activity.rank"] = ligand_dataset.index + 1
    ligand_dataset["activity.rank"] = ligand_dataset["activity.kd_nm"].rank()
    # Define active kinases
    ligand_dataset["activity.active"] = ligand_dataset["activity.kd_nm"] <= kd_cutoff
    # Add kissim data
    ligand_dataset["kissim.distance"] = [
        kissim_distance_by_kinase_name(kissim_df, kinase_name, kinase_name_query) for kinase_name_query in ligand_dataset["kinase.kinmap_name"]
    ]
    ligand_dataset["kissim.rank"] = ligand_dataset["kissim.distance"].rank()
    
    return ligand_dataset

### Example ligand dataset

In [ ]:
example_ligand_dataset = create_ligand_dataset(karaman_df, kissim_df, "Erlotinib", "EGFR", KD_CUTOFF)
example_ligand_dataset

Now, before we can use this ligand dataset for anything, we need to decide if it is good enough to be used for an analysis. What is good enough?
Use a ligand dataset only if the number of targets that fullfil the following conditions is greater than `MIN_N_TARGETS`:

- target is active w.r.t. to the ligand (in Karaman dataset; `KD_CUTOFF` defines activity)
- target is structurally covered in `kissim`

In [ ]:
example_ligand_dataset[
    (example_ligand_dataset["activity.active"]) & (example_ligand_dataset["kissim.distance"].notna())
]

### Select ligand datasets with enough coverage

Iterate over all proposed ligand-target pairs and keep only those that meet the conditions described above.

In [ ]:
ligand_targets_list_selected = []

for ligand_name, kinase_name in ligand_targets_list:
    display(Markdown(f"#### {ligand_name} / {kinase_name}"))
    example_ligand_dataset = create_ligand_dataset(karaman_df, kissim_df, ligand_name, kinase_name, KD_CUTOFF)
    a = example_ligand_dataset[
        (example_ligand_dataset["activity.active"]) & (example_ligand_dataset["kissim.distance"].notna())
    ]
    if a.shape[0] > MIN_N_TARGETS:
        display(Markdown("Include ligand dataset: YES"))
        ligand_targets_list_selected.append([ligand_name, kinase_name])
    else:
        display(Markdown("Include ligand dataset: NO"))
    display(HTML(a.to_html()))

#### Selected ligand datasets

Include the following ligand-target pairs for our ligand datasets:

In [ ]:
ligand_targets_list_selected

## Calculate `kissim` performance

How do we assess the performance of `kissim` based on our merged Karaman-`kissim` datasets (ligand datasets)?

Enrichment factor 

$EF_{x\%} = \frac{a}{b}$

- $a$: Ratio of active targets in `kissim`'s top x% ranks
- $b$: Ratio of active targets in all `kissim` ranks

In [ ]:
def enrichment(ligand_dataset, cutoff_percentage=10):
    
    kinases_total = ligand_dataset.dropna(subset=["kissim.rank"]).sort_values("kissim.distance")[1:]
    n_kinases_total = kinases_total.shape[0]
    
    n_kinases_top_x = int(n_kinases_total * cutoff_percentage / 100.0)
    kinases_top_x = kinases_total.head(n_kinases_top_x)
    
    n_active_kinases_total = kinases_total["activity.active"].sum()
    n_active_kinases_top_x = kinases_top_x["activity.active"].sum()
    
    ratio_active_kinases_identified = n_active_kinases_top_x / n_active_kinases_total
    ratio_ranked_data = n_kinases_top_x / n_kinases_total
    
    return ratio_active_kinases_identified, ratio_ranked_data

In [ ]:
def enrichment_optimal(ligand_dataset):

    n_kinases = ligand_dataset.dropna(subset=["kissim.rank"]).shape[0]
    n_active_kinases = ligand_dataset.dropna(subset=["kissim.rank"])["activity.active"].sum()

    return n_active_kinases / n_kinases

In [ ]:
def enrichment_factor(ligand_dataset, cutoff_percentage=10):
    
    ratio_active_kinases_identified, ratio_ranked_data = enrichment(ligand_dataset, cutoff_percentage)
    ef = ratio_active_kinases_identified * ratio_ranked_data
    return ef

### Test performance

In [ ]:
performances = []
enrichment_data_list = []

for ligand_name, kinase_name in ligand_targets_list_selected:
    
    ligand_dataset = create_ligand_dataset(karaman_df, kissim_df, ligand_name, kinase_name, KD_CUTOFF)
    n_actives = ligand_dataset[ligand_dataset["activity.active"]].shape[0]
    
    display(Markdown(f"#### {ligand_name} / {kinase_name}"))
    
    ef_5 = enrichment_factor(ligand_dataset, 5)
    ef_10 = enrichment_factor(ligand_dataset, 10)
    ef_20 = enrichment_factor(ligand_dataset, 20)
    
    enrichment_data = [enrichment(ligand_dataset, i) for i in range(0, 101, 1)]
    enrichment_data_list.append([ligand_name, kinase_name, enrichment_data])
    display(Markdown(f"EF5% = {ef_5}"))
    display(Markdown(f"EF10% = {ef_10}"))
    display(Markdown(f"EF20% = {ef_20}"))
    
    n_profiling_ranks = ligand_dataset["activity.rank"].notna().sum()
    n_kissim_ranks = ligand_dataset["kissim.rank"].notna().sum()
    a = ligand_dataset[ligand_dataset["activity.active"]].sort_values("activity.kd_nm")
    b = ligand_dataset.dropna(subset=["kissim.distance"]).sort_values("kissim.distance")
    n_top_kissim_kinases = 20
    
    display(Markdown(f"Show active kinases on {ligand_name} and their `kissim` ranks (in total {n_kissim_ranks} ranks):"))
    display(HTML(a.to_html()))
    display(Markdown(f"Show most similar kinases to {kinase_name} (top {n_top_kissim_kinases} `kissim` ranks) and their profiling ranks (in total {n_profiling_ranks}):"))
    display(HTML(b.head(n_top_kissim_kinases).to_html()))
    
    performances.append([ligand_name, kinase_name, ef_5, ef_10, ef_20, n_actives])

### Summarize performance

In [ ]:
cm = sns.light_palette("blue", as_cmap=True)

In [ ]:
performances_df = pd.DataFrame(performances, columns=["ligand", "kinase", "EF5%", "EF10%", "EF20%", "#actives"])
performances_df = performances_df.set_index(["ligand", "kinase"]).round(2)
performances_df.sort_values("#actives").style.background_gradient(cmap=cm, subset=["EF5%", "EF10%", "EF20%"])

In [ ]:
performances_df[["EF5%", "EF10%", "EF20%"]].transpose().plot(legend=False)

In [ ]:
performances_df[["EF5%", "EF10%", "EF20%"]].mean()

In [ ]:
enrichment_data = pd.DataFrame(enrichment_data_list, columns=["ligand", "kinase", "data"]).explode("data")
enrichment_data["x"] = enrichment_data["data"].apply(lambda x: x[1])
enrichment_data["y"] = enrichment_data["data"].apply(lambda x: x[0])
enrichment_data = enrichment_data.drop("data", axis=1)
enrichment_data

In [ ]:
data = {}
for ligand_name, data1 in enrichment_data.groupby("ligand"):
    data[ligand_name] = []
    for kinase_name, data2 in data1.groupby("kinase"):
        data_tmp = data2.set_index("x")["y"]
        data_tmp.name = kinase_name
        data[ligand_name].append(data_tmp)
    data[ligand_name] = pd.concat(data[ligand_name], axis=1)

for ligand_name, df in data.items():
    # Experimental curves
    ax = data[ligand_name].plot(title=ligand_name, ylim=(0, 1.01), xlim=(-0.01, 1))
    # Optimal curve for ligand
    ligand_dataset = create_ligand_dataset(karaman_df, kissim_df, ligand_name, kinase_name, KD_CUTOFF)
    ax.plot([0, enrichment_optimal(ligand_dataset), 1], [0, 1, 1], "--", color="k")
    # Cosmetics
    ax.set_xlabel("% ranked kissim dataset")
    ax.set_ylabel("% true active kinases identified")
    ax.set_aspect(1.0 / ax.get_data_ratio(), adjustable="box")

## Appendix